<a href="https://colab.research.google.com/github/cosmo3xplorer/BrainTumour_Detection/blob/main/Brain_Tumor_Classification_SMPL5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
import keras
import os, gc, pathlib
from sklearn.metrics import confusion_matrix
from fastai import *
from fastai.vision.all import *
from fastai.tabular.all import *
from fastai.text.all import *
#from fastai.medical.imaging import *
from fastai.vision.data import ImageDataLoaders
from fastai.vision.models import *
import torchvision.models as models
from fastai.callback.schedule import lr_find
from fastai.callback.schedule import *
from matplotlib import pyplot as plt
from fastai.imports import *
from fastai.torch_core import *
from fastai.learner import *

In [ ]:
print(os.listdir("/content/drive/MyDrive/Colab Notebooks/samples"))
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/samples/Brain_Tumor"
os.listdir(f'{DATA_DIR}')

In [ ]:
data = ImageDataLoaders.from_folder(DATA_DIR, train=".", 
                                    valid_pct=0.2,
                                    ds_tfms=aug_transforms(mult=1.0, do_flip=True, flip_vert=True, 
                                    max_warp=0, max_rotate=10.0, 
                                    max_zoom=1.1, max_lighting=0.2,
                                    p_affine=0.75, mode='bilinear',
                                    pad_mode='reflection',
                                    align_corners=True, min_scale=1.0),
                                    item_tfms=Resize(224),
                                    bs=64,
                                    val_bs=None,
                                    num_workers=0,
                                    batch_tfms=Normalize.from_stats(*imagenet_stats))

In [ ]:
data.show_batch(nrows=4, figsize=(10,10))

In [ ]:
learner = cnn_learner(data, models.resnet50, metrics=[accuracy, error_rate], cbs=[ShowGraphCallback()], model_dir="/tmp/model/")

In [ ]:
learner.lr_find()

In [ ]:
learner.model

In [ ]:
learner.fit_one_cycle(6, lr_max=slice(8.32e-4), cbs=[ShowGraphCallback()])

In [ ]:
learner.export("/content/drive/MyDrive/Colab Notebooks/pretrained_model/resnet/btc2.pkl")

In [ ]:
learner.unfreeze()

In [ ]:
learner.lr_find()

In [ ]:
learner.fit_one_cycle(6, lr_max=slice(5.754399353463668e-6), cbs=[ShowGraphCallback()])

In [ ]:
learner.fit_one_cycle(4, lr_max=slice(2.7e-5), cbs=[ShowGraphCallback()])

In [ ]:
learner.fit_one_cycle(4, lr_max=slice(2.9e-5), cbs=[ShowGraphCallback()])

In [ ]:
learner.export("/content/drive/MyDrive/Colab Notebooks/pretrained_model/resnet/final_resnet/btc_final2.pkl")

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)

In [ ]:
interp.plot_top_losses(25, figsize=(20,20))

In [ ]:
interp.plot_confusion_matrix(figsize=(8,8), dpi=60)